In [3]:
import os, sys
import scipy
import numpy as np
import ccdproc as ccdp
from astropy.io import fits
from astropy.time import Time
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from scipy.ndimage import minimum_filter1d
from astropy.table import Table, Row, Column
from scipy.ndimage.filters import percentile_filter
import astroscrappy as astrocrap
from astropy.nddata import CCDData

from matplotlib.animation import FuncAnimation
from IPython.display import HTML

plt.rcParams['font.size'] = 15

In [4]:
tics = [333680372, 146522418, 427346731, 246897668, 178947176]
parula_colors = np.load('/Users/arcticfox/parula_colors.npy', allow_pickle=True)
plot_dir = '/Users/arcticfox/Documents/youngStars/veloce/plots/'
tab = Table.read('/Users/arcticfox/Documents/youngStars/veloce/master_log.tab', format='ascii')
DISCRETE = np.load('./201108/discrete_models.npy', allow_pickle=True)
FIT_X = np.arange(300, 4000, 1)

## Get files by date

In [59]:
def get_global_files(date):
    directory = ''.join(i for i in date.split('-'))[2:]
    
    files = np.sort([os.path.join(directory,i) for i in 
                 os.listdir(directory) if i.endswith('.npy')])
    global_files = np.sort([i for i in files if 'nov' not in i])
    
    return global_files

def get_files_by_date(date):
    directory = ''.join(i for i in date.split('-'))[2:]
    files = np.sort([os.path.join(directory,i) for i in 
                 os.listdir(directory) if i.endswith('.npy')])

    if date == '2020-11-08' or date == '2020-11-09' or date == '2020-11-10':
        subdir = directory + '/masked/'
        science_files = np.sort([os.path.join(subdir, i) for i in os.listdir(subdir)])
    else:
        science_files = np.sort([i for i in files if 'nov' in i])
    
    return science_files

def get_cr_files(date):
    directory='/Users/arcticfox/Documents/youngStars/veloce/spectra/cr_masked'
    files = np.sort([os.path.join(directory, i) for i in 
                     os.listdir(directory)])
    
    day = date.split('-')[-1] + 'nov'
    
    return np.sort([i for i in files if day in i])

## Masking Cosmic Rays

In [60]:
def mask_cosmic_scum(data):
    mask, masked_data = astrocrap.detect_cosmics(data)
    return masked_data

## Modeling & removing the blaze

In [61]:
def model_flat_field(flat, order=26, size=100, percentile=55, degree=10):
    x = np.arange(0, len(flat[order]),1)
    
    # Find discontinuity in flat
    med = np.nanmedian(np.diff(flat[order]))
    std = np.nanstd(np.diff(flat[order]))

    flat_len = int(np.where(np.diff(flat[order]) >= 10*std + med)[0])
    
    # Fit first half of flat
    filt1 = percentile_filter(flat[order][:flat_len], percentile=percentile, size=size)
    fit1 = np.polyfit(x[:flat_len], filt1, deg=degree)
    model1 = np.poly1d(fit1)

    # Fit second half of flat
    filt2 = percentile_filter(flat[order][flat_len:], percentile=percentile, size=size)
    fit2 = np.polyfit(x[flat_len:], filt2, deg=degree)
    model2 = np.poly1d(fit2)
    
    model = np.append(model1(x[:flat_len]), model2(x[flat_len:]))
    return x, model

## Extracting orders from npy files

In [62]:
def extract_orders(filename, dark, order, order_offset=[0,0], fntype='science', std=2.5,
                    border_offset=[0,0]):
    """
    Inputs
    ------
    order : int
        which order to extract
    order_offset : array
        offset for the top & bottom models to extract the data
    border_offset: array
        offset for setting surrounding background to NaNs
    std : float
        masks bad pixels above certain standard deviation threshold
    """
    global DISCRETE, FIT_X

    data = np.load(filename, allow_pickle=True) + 0.0
    #data, _ = ccdp.cosmicray_lacosmic(data) # masks cosmic rays
    
    if data.shape == (4112, 4202):
        
        # extract the orders
        flux = np.zeros((DISCRETE.shape[0]-1, len(DISCRETE[0])))
        data = data - dark + 0.0
        
        # remove pesky cosmic rays n shit
        if fntype == 'science':
            #rows, cols = np.where((data > np.nanmedian(data)+std*np.nanstd(data)))
            #data[rows,cols] = np.nan
            
            for i in range(len(DISCRETE[order])):
                start = int(DISCRETE[order+1][i]+border_offset[0])
                stop  = int(DISCRETE[order+1][i]+border_offset[-1])
                data[FIT_X[i],start:stop] = np.nan
                data[FIT_X[i],start:stop] = np.nan
                
                start = int(DISCRETE[order][i]+border_offset[0])
                stop  = int(DISCRETE[order][i]+border_offset[-1])
                data[FIT_X[i],start:stop] = np.nan
                data[FIT_X[i],start:stop] = np.nan

        #for i in range(0, DISCRETE.shape[0]-1):

        top = DISCRETE[order]+order_offset[0]
        avg_width = np.abs(np.nanmedian(DISCRETE[order+1] - DISCRETE[order]))
        bottom = np.array(top + avg_width, dtype=int) - order_offset[-1]
        
        for j in range(FIT_X[0],len(FIT_X)):
            subdat = data[j, top[j]:bottom[j]]
            flux[order][j] = np.nansum(subdat)

        return data, flux
    else:
        print('Bad File: ', filename)
        return None, None

In [97]:
def which_element(element):
    if element == 'calcium':
        order = 5
        border_offset=[-3, 80]
        savedir = 'calcium_triplet/'
        return order, border_offset, savedir
    elif element == 'halpha':
        order = 26
        border_offset=[-5,25]
        savedir = 'halpha_orders/'
        return order, border_offset, savedir
    else:
        print('Element not incorporated yet')
        return None

In [91]:
def diagnosis_plot(orders, order, dat, filt):
    global FIT_X, DISCRETE
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,4))

    flattened = orders[order]/filt
    ax2.plot(flattened[1300:1600], c=parula_colors[100])
    #ax2.set_xlim(1300,1600)
    ax2.set_ylim(np.nanmin(flattened[1300:1600])-0.02,
                 np.nanmax(flattened[1300:1600])+0.02)

    ax1.imshow(dat.T, vmin=-30, vmax=150)
    ax1.plot(FIT_X, DISCRETE[order], 'k', lw=2)
    avg_width = np.abs(np.nanmedian(DISCRETE[order+1]-DISCRETE[order]))

    ax1.plot(FIT_X, DISCRETE[order]+avg_width, 'k', lw=2)
    ax1.set_xlim(1200,1900)
    ax1.set_ylim(3600,3200)
    #ax1.set_ylim(np.nanmin(DISCRETE[order])+100, 
    #             np.nanmax(DISCRETE[order])+300)
    ax1.set_xticks([])
    ax1.set_yticks([])
    plt.subplots_adjust(wspace=0.1)
    plt.show()

In [92]:
def remove_cosmic_rays(date, savedir):
    
    science_files, global_files = get_files(date)
    
    for filename in tqdm_notebook(science_files):
        fits_filename = filename[:-4] + '.fits'
        hdu = fits.open(fits_filename)
        ccd = CCDData(hdu[0].data, unit='electron')
        hdu.close()
    
        newfilename = filename.split('/')[-1][:-4]+'_maskedCRs.npy'

        if newfilename not in os.listdir(savedir):
            ccd_removed = ccdp.cosmicray_lacosmic(ccd, sigclip=5)#,
                                                  #verbose=True) # masks cosmic rays
            np.save(os.path.join(savedir, newfilename), ccd_removed.data)


In [93]:
def create_masked_files(date):
    cr_dir = '/Users/arcticfox/Documents/youngStars/veloce/spectra/cr_masked'
    remove_cosmic_rays(date, cr_dir)

## Main Function

In [94]:
%matplotlib inline

def main(date, element, plot=False, save=False, directory=None):
    global  DISCRETE, FIT_X

    order, border_offset, savedir = which_element(element)
    
    save_tab = Table(names=['Date', 'TIC', 'Filename'],
                       dtype=['U32', int, 'U32'])

    global_files = get_global_files(date)
    science_files = get_cr_files(date)
    dark_med = np.load(global_files[1], allow_pickle=True)

    if date > '2020-11-11':
        offset = 5
    else:
        offset = 0

    for i, FN in enumerate(science_files):

        f = int(FN.split('/')[-1].split('_')[0][-3:])

        which_tic = int(tab[(tab['Frame']==f) & (tab['ObsDate'] == date)]['ObjType'][0][3:])

        dat, orders = extract_orders(FN, dark_med, order,
                                     order_offset=[0,0], std=5.0,
                                     border_offset=border_offset)

        if type(dat) == np.ndarray:
            filt = percentile_filter(orders[order], percentile=60, size=300)
            flattened = orders[order]/filt
            
            if plot:
                diagnosis_plot(orders, order, dat, filt)

        newname = FN.split('.')[0].split('/')[-1] + '_{0}.npy'.format(element)

        np.save(os.path.join(savedir, newname), flattened/np.nanmedian(flattened))

        save_tab.add_row([date, which_tic, FN])
        
    if save:
        save_tab.write('{0}_{1}_orders.tab'.format(element, date), 
                        format='ascii', overwrite=True)

In [96]:
dates = np.unique(tab['ObsDate'])

for day in dates:
    main(day, element='calcium')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
